In [1]:
from theano.sandbox import cuda

Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)
/home/ubuntu/anaconda2/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)


In [206]:
from keras.applications.vgg16 import VGG16

In [2]:
import pandas as pd
import numpy as np
from utils import *
import keras
from keras.layers.normalization import BatchNormalization
import keras.layers
from keras.layers import Dropout,Dense,Activation
from keras.models import Sequential

%matplotlib inline

Using Theano backend.


In [3]:

from sklearn.cross_validation import train_test_split

In [4]:
import os,sys

In [5]:
current_dir = os.getcwd()
data_path = current_dir + '/data/Iceberg'

### Data Preprocessing

In [6]:
train_data = pd.read_json(data_path+'/data/processed/train.json')

In [7]:
train_data.head()

,band_1,band_2,id,inc_angle,is_iceberg
0,"[-27.878361, -27.15416, -28.668615, -29.537971...","[-27.154118, -29.537888, -31.0306, -32.190483,...",dfd5f913,43.9239,0
1,"[-12.242375, -14.920305, -14.920363, -12.66633...","[-31.506321, -27.984554, -26.645678, -23.76760...",e25388fd,38.1562,0
2,"[-24.603676, -24.603714, -24.871029, -23.15277...","[-24.870956, -24.092632, -20.653963, -19.41104...",58b2aaa0,45.2859,1
3,"[-22.454607, -23.082819, -23.998013, -23.99805...","[-27.889421, -27.519794, -27.165262, -29.10350...",4cfc3a18,43.8306,0
4,"[-26.006956, -23.164886, -23.164886, -26.89116...","[-27.206915, -30.259186, -30.259186, -23.16495...",271f93f4,35.6256,0


In [16]:
len(train_data)

1604

In [22]:
len(train_data['band_2'][1])

5625

In [8]:
X_band1 = np.array([np.array(band).reshape(75,75) for band in train_data['band_1']] )

In [9]:
X_band2 = np.array([np.array(band).reshape(75,75) for band in train_data["band_2"]])

In [10]:
train_dt = np.concatenate([X_band1[:,:,:,np.newaxis],X_band2[:,:,:,np.newaxis],((X_band1+X_band2)/2)[:,:,:,np.newaxis]],axis=-1)

In [44]:
#Testing axis swap for theano
train_dt_Swap = np.concatenate([((X_band1+X_band2)/2)[:,:,:,np.newaxis],X_band1[:,:,:,np.newaxis],X_band2[:,:,:,np.newaxis]],axis=-1)

In [11]:
train_dt.shape

(1604, 75, 75, 3)

In [45]:
train_dt_Swap.shape

(1604, 75, 75, 3)

In [12]:
train_incd_angle = train_data['inc_angle'].replace('na',0)

In [13]:
train_incd_angle = np.array(train_incd_angle)
train_incd_angle

array([ 43.9239,  38.1562,  45.2859, ...,   0.    ,   0.    ,   0.    ])

In [14]:
train_Y = np.array(train_data['is_iceberg'])

### Test Data

In [15]:
test_data = pd.read_json(data_path+'/data/processed/test.json')

In [16]:
test_band1 = np.array([np.array(band).reshape(75,75) for band in test_data['band_1']])

In [17]:
test_band1.shape

(8424, 75, 75)

In [18]:
test_band2 = np.array([np.array(band).reshape(75,75) for band in test_data['band_2']])

In [19]:
test_band2.shape

(8424, 75, 75)

In [20]:
test_dt = np.concatenate([test_band1[:,:,:,np.newaxis],test_band2[:,:,:,np.newaxis],((test_band1 + test_band2)/2)[:,:,:,np.newaxis]],axis = -1)

In [115]:
test_data.head()

,band_1,band_2,id,inc_angle
0,"[-15.863251, -15.201077, -17.887735, -19.17248...","[-21.629612, -21.142353, -23.908337, -28.34524...",5941774d,34.966400
1,"[-26.0589694977, -26.0589694977, -26.058969497...","[-25.7542076111, -25.7542076111, -25.754207611...",4023181e,32.615072
2,"[-14.1410999298, -15.0642414093, -17.375520706...","[-14.745639801, -14.5904102325, -14.3626976013...",b20200e4,37.505433
3,"[-12.167478, -13.706167, -16.54837, -13.572674...","[-24.32222, -26.375538, -24.096739, -23.8769, ...",e7f018bb,34.473900
4,"[-23.3745937347, -26.0271816254, -28.121963501...","[-25.7223434448, -27.0115776062, -23.149162292...",4371c8c3,43.918874


In [22]:
test_inc_angle = test_data['inc_angle'].replace('na',0)

In [23]:
test_inc_angle =np.array(test_inc_angle )

In [24]:
test_inc_angle .shape

(8424,)

In [25]:
X_train,X_valid,X_train_angle,X_valid_angle,Y_train,Y_valid = train_test_split(train_dt,train_incd_angle,train_Y,random_state =123,train_size=0.75)

In [70]:
#after arranging axis and making channel first test,train split
X_train_channel_First,X_valid_CHan_first,X_train_angle_Chan_first,X_valid_angle_chan_first,Y_train_chan_first,Y_valid_chan_first= train_test_split(X_t_n,train_incd_angle,train_Y,random_state =123,train_size=0.75)

In [53]:

test_dt_channel_first = np.concatenate([((test_band1 + test_band2)/2)[:,:,:,np.newaxis],test_band1[:,:,:,np.newaxis],test_band2[:,:,:,np.newaxis]],axis = -1)

In [ ]:
X_train = np.roll()X_train.reshape()

In [62]:
X_test_n = []
for image in test_dt_channel_first:
    image_c = np.swapaxes(np.swapaxes(image,1,2),0,1)
    X_test_n.append(image_c)
    

In [63]:
X_test_n = np.array(X_test_n)

In [64]:
X_test_n.shape

(8424, 3, 75, 75)

In [68]:
X_t_n = np.array(X_NewT)

In [69]:
X_t_n.shape

(1604, 3, 75, 75)

In [66]:
X_NewT = []
for image in train_dt_Swap:
    image_convert=np.swapaxes(np.swapaxes(image, 1, 2), 0, 1)
    X_NewT.append(image_convert)



### Preparing the model

In [27]:
imgGen = keras.preprocessing.image.ImageDataGenerator()

In [28]:
Batches = imgGen.flow(X_train,Y_train,)

In [29]:
model = Sequential()

In [103]:
model.add(Conv2D(128,3,3,activation='relu',input_shape=(75,75,3)))
model.add(BatchNormalization(axis=1))
model.add(MaxPooling2D())
model.add(Dropout(0.2))


In [104]:
model.add(Conv2D(256,3,3,activation='relu'))
model.add(BatchNormalization(axis=1))
model.add(MaxPooling2D())
model.add(Dropout(0.2))

In [ ]:
model.add(Dense())

In [87]:
model = Sequential([BatchNormalization(axis=1,input_shape=(3,75,75)),
                    Conv2D(64,3,3,activation='relu'),
                    BatchNormalization(axis=1),
                    MaxPooling2D(),
                    Dropout(0.2),    
                    Conv2D(128,3,3,activation='relu'),
                    BatchNormalization(axis=1),
                    MaxPooling2D(),
                    Dropout(0.4),  
                    Conv2D(256,3,3,activation='relu'),
                    BatchNormalization(axis=1),
                    MaxPooling2D(),
                    Dropout(0.5), 
                    Flatten(),
                    Dense(128,activation='relu'),
                    BatchNormalization(),
                    Dense(128,activation='relu'),
                    BatchNormalization(),
                    Dense(1,activation='sigmoid')
                   ])

In [72]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
batchnormalization_25 (BatchNorma(None, 3, 75, 75)     6           batchnormalization_input_7[0][0] 
____________________________________________________________________________________________________
convolution2d_9 (Convolution2D)  (None, 64, 73, 73)    1792        batchnormalization_25[0][0]      
____________________________________________________________________________________________________
batchnormalization_26 (BatchNorma(None, 64, 73, 73)    128         convolution2d_9[0][0]            
____________________________________________________________________________________________________
maxpooling2d_8 (MaxPooling2D)    (None, 64, 36, 36)    0           batchnormalization_26[0][0]      
___________________________________________________________________________________________

In [99]:
model.compile('RMSprop',loss='binary_crossentropy',metrics=['accuracy'])

In [75]:
model.fit(X_train_channel_First,Y_train_chan_first,nb_epoch=1,validation_data=[X_valid_CHan_first,Y_valid_chan_first])

Train on 1203 samples, validate on 401 samples
Epoch 1/1
1203/1203 [==============================] - 5s - loss: 0.1152 - acc: 0.9659 - val_loss: 1.8922 - val_acc: 0.6584


In [111]:
model.optimizer.lr= 0.0001

In [112]:
model.fit(X_train_channel_First,Y_train_chan_first,nb_epoch=10,validation_data=[X_valid_CHan_first,Y_valid_chan_first])

Train on 1203 samples, validate on 401 samples
Epoch 1/10
1203/1203 [==============================] - 7s - loss: 0.0819 - acc: 0.9717 - val_loss: 0.5851 - val_acc: 0.8080
Epoch 2/10
1203/1203 [==============================] - 7s - loss: 0.0803 - acc: 0.9676 - val_loss: 0.4773 - val_acc: 0.8479
Epoch 3/10
1203/1203 [==============================] - 7s - loss: 0.0762 - acc: 0.9717 - val_loss: 0.4449 - val_acc: 0.8778
Epoch 4/10
1203/1203 [==============================] - 7s - loss: 0.0477 - acc: 0.9817 - val_loss: 0.5092 - val_acc: 0.8479
Epoch 5/10
1203/1203 [==============================] - 7s - loss: 0.0650 - acc: 0.9800 - val_loss: 0.5651 - val_acc: 0.8678
Epoch 6/10
1203/1203 [==============================] - 7s - loss: 0.0462 - acc: 0.9817 - val_loss: 0.5946 - val_acc: 0.8554
Epoch 7/10
1203/1203 [==============================] - 7s - loss: 0.0396 - acc: 0.9875 - val_loss: 0.6447 - val_acc: 0.8479
Epoch 8/10
1203/1203 [==============================] - 7s - loss: 0.0663 - ac

In [134]:
prediction = model.predict(X_test_n)

In [126]:
np.array(test_data[['id']]).shape

(8424, 1)

In [ ]:
prediction = np.stack([np.array(test_data[['id']]),prediction])

In [136]:
Sample_Sub01

,0
0,4.138977e-04
1,9.998083e-01
2,3.043057e-02
3,9.997153e-01
4,9.992154e-01
5,9.967903e-01
6,8.333768e-01
7,9.999827e-01
8,1.292415e-05
9,5.922703e-06


In [142]:
Sample_Sub01 = pd.DataFrame(prediction,columns=['is_iceberg'])

In [167]:
Sample_Sub01

,id,is_iceberg
0,5941774d,4.138977e-04
1,4023181e,9.998083e-01
2,b20200e4,3.043057e-02
3,e7f018bb,9.997153e-01
4,4371c8c3,9.992154e-01
5,a8d9b1fd,9.967903e-01
6,29e7727e,8.333768e-01
7,92a51ffb,9.999827e-01
8,c769ac97,1.292415e-05
9,aee0547d,5.922703e-06


In [166]:
Sample_Sub01.insert(0,'id',test_data['id'])

ValueError: cannot insert id, already exists

#### First Submission just to test the waters. Oye Oye Oye...I am coming biat

In [177]:
sub003 = "/resultss/sub03.csv"

In [163]:
pwd

u'/home/ubuntu/courses/deeplearning1/nbs'

In [178]:
Sample_Sub01.to_csv("/home/ubuntu/courses/deeplearning1/nbs/data/Iceberg" + sub003)

In [179]:
from IPython.display import FileLink
%cd ~

/home/ubuntu


In [150]:
FileLink(firstsub)

/home/ubuntu/courses/deeplearning1/nbs/data/Iceberg/resultss/sub01.csv

In [181]:
%cd ~/courses/deeplearning1/nbs

/home/ubuntu/courses/deeplearning1/nbs


In [182]:
FileLink('data/Iceberg'+sub003)

/home/ubuntu/courses/deeplearning1/nbs/data/Iceberg/resultss/sub03.csv

### Transfer Learning (VGG)

In [183]:
import vgg16


In [187]:
from vgg16 import Vgg16
vgg= Vgg16()
model= vgg.model

In [188]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lambda_2 (Lambda)                (None, 3, 224, 224)   0           lambda_input_2[0][0]             
____________________________________________________________________________________________________
zeropadding2d_14 (ZeroPadding2D) (None, 3, 226, 226)   0           lambda_2[0][0]                   
____________________________________________________________________________________________________
convolution2d_27 (Convolution2D) (None, 64, 224, 224)  1792        zeropadding2d_14[0][0]           
____________________________________________________________________________________________________
zeropadding2d_15 (ZeroPadding2D) (None, 64, 226, 226)  0           convolution2d_27[0][0]           
___________________________________________________________________________________________

In [189]:
layer = model.layers

In [192]:
last_idx = [index for index,value in enumerate(layer)
               if type(value) is Convolution2D][-1]

In [194]:
conv_layers = layer[:last_idx+1]

In [195]:
new_model = Sequential(conv_layers)

In [197]:
new_model.compile('RMSprop',loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
new_model.fit(X_train_channel_First,Y_train_chan_first,nb_epoch=1,validation_data=[X_valid_CHan_first,Y_valid_chan_first])

In [209]:
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
from keras.layers import Input, Flatten, Dense
from keras.models import Model

In [213]:
model_vgg16 = keras.applications.vgg16.VGG16(include_top=False, weights='imagenet', input_tensor=None)


In [212]:
input = Input(shape=(3,75,75))

In [214]:
out_vgg16 = model_vgg16(input)

In [223]:
#Adding FC layers 
x = Flatten()(out_vgg16)
x= Dense(4096,activation='relu')(x)
x = Dense(4096,activation='relu')(x)
x = Dense(1,activation='sigmoid')(x)

In [224]:
my_vgg16 = Model(input = input,output=x)



In [225]:
my_vgg16.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_2 (InputLayer)             (None, 3, 75, 75)     0                                            
____________________________________________________________________________________________________
model_2 (Model)                  multiple              14714688    input_2[0][0]                    
____________________________________________________________________________________________________
flatten_14 (Flatten)             (None, 2048)          0           model_2[1][0]                    
____________________________________________________________________________________________________
dense_40 (Dense)                 (None, 4096)          8392704     flatten_14[0][0]                 
___________________________________________________________________________________________

In [240]:
my_vgg16.compile('RMSprop',loss='binary_crossentropy',metrics =['accuracy'],)

In [246]:
my_vgg16.fit(X_train_channel_First,Y_train_chan_first,nb_epoch=5,validation_data=[X_valid_CHan_first,Y_valid_chan_first])

Train on 1203 samples, validate on 401 samples
Epoch 1/5
1203/1203 [==============================] - 19s - loss: 7.3020 - acc: 0.5470 - val_loss: 8.3605 - val_acc: 0.4813
Epoch 2/5
1203/1203 [==============================] - 19s - loss: 7.3020 - acc: 0.5470 - val_loss: 8.3605 - val_acc: 0.4813
Epoch 3/5
1203/1203 [==============================] - 19s - loss: 7.3020 - acc: 0.5470 - val_loss: 8.3605 - val_acc: 0.4813
Epoch 4/5
1203/1203 [==============================] - 19s - loss: 7.3020 - acc: 0.5470 - val_loss: 8.3605 - val_acc: 0.4813
Epoch 5/5
1203/1203 [==============================] - 19s - loss: 7.3020 - acc: 0.5470 - val_loss: 8.3605 - val_acc: 0.4813


In [245]:
model.optimizer.lr = 0.0001

### Overfitting so let's generate more data

#### Data Augmentation